In [ ]:
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px

from eda_utils import (
    read_scip_log,
    format_log_df,
    operations_plots,
    objective_function_plots,
    get_month_difference,
    get_revenue_before_eow,
)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

%load_ext autoreload
%autoreload 2


In [ ]:
experiment = "2019_24periods_12eow_real_prices"

In [ ]:
with open("../lp_logs/experiments_results.json") as json_file:
    experiment_results = json.load(json_file)
with open("../lp_logs/params.json") as json_file:
    PARAMS = json.load(json_file)


Log_df = read_scip_log(f"../lp_logs/{experiment}.log")
Log_df = format_log_df(Log_df)
periods_before_eow = get_month_difference(
    experiment_results[experiment]["fecha_inicio"],
    experiment_results[experiment]["fecha_fin_ejercicio"],
)

In [ ]:
Log_df = read_scip_log(f"../lp_logs/{experiment}.log")

# Log_df['unit_impact_on_obj_func'] = Log_df.unit_impact_on_obj_func.astype(float)
# Log_df.loc[Log_df.unit_impact_on_obj_func > 1000]

Log_df = format_log_df(Log_df)
Log_df.loc[Log_df.impact_on_obj_func > 1000000]

In [ ]:
operations_plots(
    Log_df,
    periodos_modelo=experiment_results[experiment]["periodos_modelo"],
    periods_before_eow=periods_before_eow,
)

objective_function_plots(
    Log_df,
    periodos_modelo=experiment_results[experiment]["periodos_modelo"],
    periods_before_eow=periods_before_eow,
)

In [ ]:
experiment_results[experiment]["lp_revenue_before_eow"] = get_revenue_before_eow(
    Log_df, periods_before_eow, PARAMS, experiment_results, experiment
)

experiment_results[experiment]["lp_revenue_eow"] = Log_df["impact_on_obj_func"].sum()

In [ ]:
# Create DataFrame
data = {
    "type": ["LP", "LP", "LP", "Business", "Business", "Business"],
    "desc": [
        "lp_revenue_eow",
        "lp_stock_history_cost",
        "lp_revenue_before_eow",
        "stock_value",
        "sales_value",
        "grand_total",
    ],
    "value": [
        experiment_results[experiment]["lp_revenue_eow"],
        experiment_results[experiment]["lp_stock_history_cost"],
        experiment_results[experiment]["lp_revenue_before_eow"],
        experiment_results[experiment]["business_results"]["stock_value"],
        experiment_results[experiment]["business_results"]["sales_value"],
        experiment_results[experiment]["business_results"]["grand_total"],
    ],
}

df = pd.DataFrame(data)
df